In [3]:
#importation des bibliothèques nécessaires
import selenium
import re
from selenium import webdriver

In [7]:
def recup_bdd(id_chaine) :
    
    #chaine = chaine.replace(' ' , '')
    #initialisation webdriver
    IS_LINUX = False
    chrome = webdriver.Chrome(executable_path="./chromedriver")
    
    #récuparation des données 
    bdd = []
    lien = 'https://www.youtube.com/channel/{}/playlists?view=1&sort=dd&shelf_id=0'.format(id_chaine)
    chrome.get(lien)
    chrome.refresh()
    tag = chrome.find_elements_by_css_selector('a')
    l_url = []
    l_title = []
    for a in tag:
        url = a.get_attribute('href')
        try:
            if url.startswith("https://www.youtube.com/playlist?"):
                l_url.append(url)
            if url.startswith("https://www.youtube.com/watch?"):
                l_title.append(a.get_attribute('title'))
        except:
            pass
        
    l_title = list(filter(None,l_title))

    for idx,i in enumerate(l_url) :
        try :
            chrome.get(i)
            tag = chrome.find_element_by_id("stats")
            res = []
            for j in tag.find_elements_by_tag_name('span'):
                res.append(j.text)
            nbr_videos = res[0]
            try :
                last_modif = res[2]+res[3]+res[4]
                last_modif = last_modif.replace('Mise à jour il y a','')
                
            except :
                last_modif = res[2]+res[3]
                last_modif = last_modif.replace('Dernière modification le','')
                
            vues = chrome.find_element_by_xpath("//yt-formatted-string[2]").text
            vues = vues.replace('\u202f','')
            bdd.append({'playlist' : l_title[idx],
                    'nbr_videos' : nbr_videos,
                    'derniere_MAJ' : last_modif,
                    'vues' : vues
                   })
        except : None
            
    return bdd

In [8]:
squeezie = recup_bdd('UCWeg2Pkate69NFdBeuRFTAw')

In [9]:
squeezie

[{'playlist': 'Collections Yoko',
  'nbr_videos': '4',
  'derniere_MAJ': '26 août 2020',
  'vues': '18761 vues'},
 {'playlist': 'Activités manuelles',
  'nbr_videos': '13',
  'derniere_MAJ': '17 avr. 2020',
  'vues': '462170 vues'},
 {'playlist': 'RÉACTION',
  'nbr_videos': '23',
  'derniere_MAJ': '21 juin 2020',
  'vues': '917900 vues'},
 {'playlist': 'THÉORIE DU COMPLOT',
  'nbr_videos': '5',
  'derniere_MAJ': '30 déc. 2018',
  'vues': '649101 vues'},
 {'playlist': 'Thread horreur',
  'nbr_videos': '26',
  'derniere_MAJ': '11 déc. 2020',
  'vues': '13299876 vues'},
 {'playlist': 'VOYAGES (VLOGS)',
  'nbr_videos': '7',
  'derniere_MAJ': '3 nov. 2018',
  'vues': '263898 vues'},
 {'playlist': "'OXYZ' mon 1er album 🎵",
  'nbr_videos': '5',
  'derniere_MAJ': '7 déc. 2020',
  'vues': '3290091 vues'},
 {'playlist': "Outlast 2 (let's play)",
  'nbr_videos': '6',
  'derniere_MAJ': '4 juil. 2017',
  'vues': '1436840 vues'},
 {'playlist': "The Last Guardian (let's play)",
  'nbr_videos': '8',
 